In [1]:
import cv2
import imutils
import numpy as np
import keras
from keras.models import load_model
from skimage.io import imread
from skimage.transform import resize


Using TensorFlow backend.


In [2]:
model = load_model('Project7.h5')
model.compile(loss = keras.losses.categorical_crossentropy, optimizer=keras.optimizers.Adam(),
              metrics=['accuracy'])
bg = None

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.cast instead.


In [3]:
def run_avg(image, aWeight):
    global bg
    if bg is None:
        bg = image.copy().astype("float")
        return
    cv2.accumulateWeighted(image, bg, aWeight)

def segment(image, threshold=25):
    global bg
    diff = cv2.absdiff(bg.astype("uint8"), image)
    thresholded = cv2.threshold(diff,
                                threshold,
                                255,
                                cv2.THRESH_BINARY)[1]

    # get the contours in the thresholded image
    _,cnts,_= cv2.findContours(thresholded.copy(),
                                    cv2.RETR_EXTERNAL,
                                    cv2.CHAIN_APPROX_SIMPLE)

    # return None, if no contours detected
    if len(cnts) == 0:
        return
    else:
        # based on contour area, get the maximum contour which is the hand
        segmented = max(cnts, key=cv2.contourArea)
        return (thresholded, segmented)
    
if __name__ == "__main__":
    aWeight = 0.5

    camera = cv2.VideoCapture(0)

    # region of interest (ROI) coordinates
    top, right, bottom, left = 25, 350, 225, 590

    # initialize num of frames
    num_frames = 0
    while(True):
        # get the current frame
        (grabbed, frame) = camera.read()

        # resize the frame
        frame = imutils.resize(frame, width=700)

        # flip the frame so that it is not the mirror view
        frame = cv2.flip(frame, 1)

        # clone the frame
        clone = frame.copy()

        # get the height and width of the frame
        (height, width) = frame.shape[:2]

        # get the ROI
        roi = frame[top:bottom, right:left]

        # convert the roi to grayscale and blur it
        gray = cv2.cvtColor(roi, cv2.COLOR_BGR2GRAY)
        gray = cv2.GaussianBlur(gray, (7, 7), 0)

        # to get the background, keep looking till a threshold is reached
        # so that our running average model gets calibrated
        if num_frames < 30:
            run_avg(gray, aWeight)
        else:
            # segment the hand region
            hand = segment(gray)

            # check whether hand region is segmented
            if hand is not None:
                # if yes, unpack the thresholded image and
                # segmented region
                (thresholded, segmented) = hand

                # draw the segmented region and display the frame
                cv2.drawContours(clone, [segmented + (right, top)], -1, (0, 0, 255))
                img = resize(thresholded,(128,128,3)) 
                img = np.expand_dims(img,axis=0)
                if(np.max(img)>1):
                    img = img/255.0
                prediction = model.predict_classes(img)
                print(prediction)
                index=np.array(["hello","one","three","victory"])
                print(index[prediction])
                cv2.imshow("Thresholded", thresholded)
                
                

        # draw the segmented hand
        cv2.rectangle(clone, (left, top), (right, bottom), (0,255,0), 2)

        # increment the number of frames
        num_frames += 1

        # display the frame with segmented hand
        cv2.imshow("Video Feed", clone)

        # observe the keypress by the user
        keypress = cv2.waitKey(1) & 0xFF

        # if the user pressed "q", then stop looping
        if keypress == ord("q"):
            break

camera.release()
cv2.destroyAllWindows()

C:\Users\liki\Anaconda3\lib\site-packages\skimage\transform\_warps.py:105: UserWarning: The default mode, 'constant', will be changed to 'reflect' in skimage 0.15.
  warn("The default mode, 'constant', will be changed to 'reflect' in "
C:\Users\liki\Anaconda3\lib\site-packages\skimage\transform\_warps.py:110: UserWarning: Anti-aliasing will be enabled by default in skimage 0.15 to avoid aliasing artifacts when down-sampling images.
  warn("Anti-aliasing will be enabled by default in skimage 0.15 to "


[3]
['victory']
[3]
['victory']
[3]
['victory']
[3]
['victory']
[3]
['victory']
[3]
['victory']
[1]
['one']
[3]
['victory']
[0]
['hello']
[2]
['three']
[3]
['victory']
[3]
['victory']
[1]
['one']
[1]
['one']
[3]
['victory']
[3]
['victory']
[3]
['victory']
[3]
['victory']
[3]
['victory']
[3]
['victory']
[3]
['victory']
[0]
['hello']
[0]
['hello']
[0]
['hello']
[0]
['hello']
[0]
['hello']
[0]
['hello']
[3]
['victory']
[3]
['victory']
[3]
['victory']
[3]
['victory']
[3]
['victory']
[3]
['victory']
[3]
['victory']
[3]
['victory']
[3]
['victory']
[3]
['victory']
[3]
['victory']
[3]
['victory']
[3]
['victory']
[3]
['victory']
[3]
['victory']
[3]
['victory']
[3]
['victory']
[3]
['victory']
[3]
['victory']
[3]
['victory']
[3]
['victory']
[3]
['victory']
[3]
['victory']
[3]
['victory']
[3]
['victory']
[3]
['victory']
[3]
['victory']
[3]
['victory']
[3]
['victory']
[3]
['victory']
[3]
['victory']
[3]
['victory']
[3]
['victory']
[3]
['victory']
[3]
['victory']
[3]
['victory']
[3]
['victory']
[3]
